In [7]:
def minimumSteps(loggedMoves):
    stack = []
    
    for move in loggedMoves:
        if "../" in move:
            # move to parent
            if stack:
                stack.pop()
        elif "./" in move:
            # remain
            continue
        else:
            # move to child
            folder_name = move.strip("/")
            stack.append(folder_name)

    # num moves required
    return len(stack)

# Example usage
loggedMoves = ['x/', '../', 'y/', 'z/']
result = minimumSteps(loggedMoves)
print(result)

2


In [ ]:
/*
Enter your query below.
Please append a semicolon ";" at the end of the query
*/

select 'customer' as category, id, customer_name
from customer
where id not in (select distinct customer_id from invoice)

UNION

select 'product' as category, id, product_name
from product
where id not in (select distinct product_id from invoice_item);

In [ ]:
from pyspark.sql import SparkSession, DataFrame, Window
from main.base import PySparkJobInterface
import pyspark.sql.functions as F


class PySparkJob(PySparkJobInterface):

    def init_spark_session(self) -> SparkSession:
        return SparkSession.builder.appName('Banking Data Mining').master('local').getOrCreate()
        

    def extract_valid_transactions(self, accounts: DataFrame, transactions: DataFrame) -> DataFrame:
        #join trans & accounts
        joined = transactions.join(accounts, transactions.toAccountNumber == accounts.accountNumber, "inner")
        
        # filter for valid
        valid = joined.filter(col("transferAmount") <= col("balance"))

        return valid
        ...

    def distinct_transactions(self, transactions: DataFrame) -> int:
        #select distinct trans
        transCount = transactions.select("fromAccountNumber").distinct().count()

        return transCount
        ...

    def transactions_per_account(self, transactions: DataFrame) -> DataFrame:
        #group
        countAccount = transactions.groupBy("fromAccountNumber").agg(count(transactionId).alias("transactionCount"))

        #ordering to help rank
        sortedTrans = countAccount.orderBy(desc("transactionCount"))

        #top 10
        top10 = sortedTrans.limit(10)

        return top10 
        ...
